In [1]:
import pandas as pd
import datetime
import os
import pytz
import json
print(pd.__version__)
utc = pytz.utc

datDir = "./oura/oura_2021-08-10T02-14-44.json"


dataFileColumns = ["timestamp", "stage", "endTimestamp"]

outputFileName = "ouraRingSleepStages.parquet.gzip"

#get list of files in current directroy
files = [f for f in os.listdir("./") if os.path.isfile(f)]


#if the output file isn't already made, make it
if not (outputFileName in files):
    print("making new df")
    emptydf = pd.DataFrame(columns = dataFileColumns)
    emptydf.set_index('timestamp',inplace=True)
    print(emptydf)
        
    #save as a parquet file
    emptydf.to_parquet(outputFileName, compression='gzip') 


df = pd.read_parquet(outputFileName)

print(df.head(100))

with open(datDir) as f:
  jsonData = json.load(f)


1.2.4
making new df
Empty DataFrame
Columns: [stage, endTimestamp]
Index: []
Empty DataFrame
Columns: [stage, endTimestamp]
Index: []


In [2]:
def ouraSleepStageToStandard(ouraNum):
    if ouraNum == 4: #awake
        return 0
    if ouraNum == 2: #light sleep
        return 1
    if ouraNum == 1: #deep sleep
        return 2
    if ouraNum == 3: #REM sleep
        return 3
    

In [3]:

#seperate out the list of sleep data
for SleepJSON in jsonData['sleep']:
    startTime = datetime.datetime.fromisoformat(SleepJSON['bedtime_start']).astimezone(utc)
    stageStartTime = startTime

    hypnogram_5min = SleepJSON['hypnogram_5min']

    toadd = []
    #toadd.append(stageStartTime)
    toadd.append(ouraSleepStageToStandard(int(hypnogram_5min[0])))

    for i in range(1, len(hypnogram_5min)):
        if(hypnogram_5min[i-1] != hypnogram_5min[i]):
            #save the current time as the end of the last stage
            toadd.append(startTime + datetime.timedelta(minutes=i))
            #add to the df
            df.loc[stageStartTime] = toadd
            #reset toadd
            toadd = []
            #set the current time to the start of the next one
            stageStartTime = startTime + datetime.timedelta(minutes=i)
            #toadd.append(stageStartTime)
            #set the lable of the next stage to the value of the current index

            toadd.append(ouraSleepStageToStandard(int(hypnogram_5min[i])))
    #set the last end time to the last toadd
    toadd.append(startTime + datetime.timedelta(minutes=len(hypnogram_5min)))
    #add it to the df
    df.loc[stageStartTime] = toadd

#sperate out the bedtime_start and the hypnogram_5min

#parse it out in a very similar way to the withings data

In [ ]:
print(df)

                                          timestamp stage  \
index                                                       
2020-10-15 08:03:59+00:00 2020-10-15 08:03:59+00:00     0   
2020-10-15 08:04:59+00:00 2020-10-15 08:04:59+00:00     1   
2020-10-15 08:05:59+00:00 2020-10-15 08:05:59+00:00     0   
2020-10-15 08:06:59+00:00 2020-10-15 08:06:59+00:00     1   
2020-10-15 08:08:59+00:00 2020-10-15 08:08:59+00:00     2   
...                                             ...   ...   
2021-08-09 09:57:48+00:00 2021-08-09 09:57:48+00:00     1   
2021-08-09 10:07:48+00:00 2021-08-09 10:07:48+00:00     2   
2021-08-09 10:08:48+00:00 2021-08-09 10:08:48+00:00     1   
2021-08-09 10:13:48+00:00 2021-08-09 10:13:48+00:00     3   
2021-08-09 10:16:48+00:00 2021-08-09 10:16:48+00:00     0   

                                       endTimestamp  
index                                                
2020-10-15 08:03:59+00:00 2020-10-15 08:04:59+00:00  
2020-10-15 08:04:59+00:00 2020-10-15 08:05:5

In [ ]:
df.sort_index(inplace=True)
df.to_parquet(outputFileName, compression='gzip') 